This data processing was done using Google Colab (to load file directly from Google Drive)

### Getting the file from gdrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/MyDrive/edu-data")
!ls

video_with_channelcountry.csv.zip


In [5]:
!unzip "/content/drive/MyDrive/edu-data/video_with_channelcountry.csv.zip"

Archive:  /content/drive/MyDrive/edu-data/video_with_channelcountry.csv.zip
  inflating: video_with_channelcountry.csv  
  inflating: __MACOSX/._video_with_channelcountry.csv  


### Read file and get video data with country attributes

In [8]:
# Install required packages
! pip3 install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.6 MB/s eta 0:00:00


In [12]:
import pandas as pd
import csv
import re
import matplotlib.pyplot as plt
from rapidfuzz import fuzz, process
from tqdm import tqdm

In [11]:
# Extract video data with country field (removing those without any country information)
df_video_with_country = pd.read_csv('/content/drive/MyDrive/edu-data/video_with_channelcountry.csv')
df_video_with_country = df_video_with_country[df_video_with_country['country'].notnull()]

<ipython-input-11-896327dcbe5a>:2: DtypeWarning: Columns (7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_video_with_country = pd.read_csv('/content/drive/MyDrive/edu-data/video_with_channelcountry.csv')


In [13]:
len(df_video_with_country)

1911894

So at this starting point we have around 2 million videos :o

In [18]:
# Get final STEM keywords list
!curl -o final_keywords.txt https://raw.githubusercontent.com/epfl-ada/ada-2024-project-adacadabra2048/refs/heads/main/src/scripts/keyword_generator/final_keywords.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 93597  100 93597    0     0   159k      0 --:--:-- --:--:-- --:--:--  159k


First we "flatten" the keyword hierarchy to extract the list of unique STEM keywords, which will later be used to match with our video tags for detecting STEM content.

In [19]:
def extract_keywords(filepath):
    """
    Extracts unique keywords from a file with the format "x_keywords: [...]".
    """
    keywords = {}
    with open(filepath, 'r') as file:
        for line in file:
            match = re.match(r"(\w+_keywords)\s*=\s*\[(.*?)\]", line, re.DOTALL)
            if match:
                category = match.group(1)
                keyword_str = match.group(2)
                # Handle both single and double quotes, special characters,
                # remove extra whitespace, and filter out empty strings
                # The change is in the regex:
                keyword_list = [''.join(k).strip().strip("'").strip('"').strip("’") for k in re.findall(r"'([^']*)'|[\"]([^\"]*)[\"]|’([^’]*)’", keyword_str)]
                keyword_list = [keyword for keyword in keyword_list if keyword]
                keywords[category] = keyword_list

    # Flatten the dictionary and remove duplicates
    all_keywords = []
    for category, wordlist in keywords.items():
        all_keywords.extend(wordlist)

    return list(set(all_keywords))

stem_keywords = extract_keywords('final_keywords.txt')
len(stem_keywords)

2008

In [20]:
# Sanity Check:
keywords_to_check = ["python", "java", "circuits", "language"]
for keyword in keywords_to_check:
  if keyword.capitalize() in stem_keywords:
    print(f"'{keyword}' found in stem_keywords")
  else:
    print(f"'{keyword}' not found in stem_keywords")

'python' found in stem_keywords
'java' found in stem_keywords
'circuits' found in stem_keywords
'language' not found in stem_keywords


Now we fuzzy match the tags of each video with our unique STEM keyword list that we've just constructed. We define a heuristic threshold of 50%, where a video with more than half of their tags being considered as STEM keywords will be considered as STEM videos.

In [25]:
# Pre-process keywords (convert to lowercase)
stem_keywords_lower = [keyword.lower() for keyword in stem_keywords]

def is_stem_video(tags):
    """Checks if a video is STEM based on its tags."""
    if pd.isna(tags):
        return False

    video_tags = tags.lower().split(',')  # Convert tags to lowercase for efficient matching
    matched_tags = 0
    for video_tag in video_tags:
        # Using rapidfuzz's process.extractOne for faster fuzzy matching
        match = process.extractOne(video_tag.strip(), stem_keywords_lower, scorer=fuzz.ratio, score_cutoff=70)
        if match:
            matched_tags += 1

    return matched_tags >= len(video_tags) / 2

# Initialize keyword counts with lowercase keys
keyword_counts = {keyword.lower(): 0 for keyword in stem_keywords}

# Apply the is_stem_video function using Pandas apply for vectorization
# with tqdm progress bar
tqdm.pandas(desc="Processing is_stem")
df_video_with_country['is_stem'] = df_video_with_country['tags'].progress_apply(is_stem_video)

# Update keyword counts (this part is still iterative, but it's unavoidable)
for index, row in tqdm(df_video_with_country[df_video_with_country['is_stem']].iterrows(), total=len(df_video_with_country[df_video_with_country['is_stem']])):
    video_tags = row['tags'].lower().split(',')
    for video_tag in video_tags:
        match = process.extractOne(video_tag.strip(), stem_keywords_lower, scorer=fuzz.ratio, score_cutoff=70)
        if match:
            keyword_counts[match[0]] += 1

# Print the number of videos with is_stem = True
stem_video_count = df_video_with_country['is_stem'].sum()
print(f"Number of STEM videos: {stem_video_count}")

100%|██████████| 56765/56765 [01:50<00:00, 512.02it/s]

Number of STEM videos: 56765


Out of around 2 million educational videos (with country data), more than 56,000 of them are considered STEM videos (~3%) by our detection method.

In [26]:
# Save the DataFrame with the 'is_stem' column to a new CSV file
df_video_with_country.to_csv('video_with_country_and_stem.csv', index=False)

# Save the keyword counts to a text file
with open('keyword_counts.txt', 'w') as f:
    for keyword, count in keyword_counts.items():
        f.write(f'{keyword},{count}\n')